<a href="https://colab.research.google.com/github/anshupandey/Generative-AI-for-Professionals/blob/main/langchain-course/01_Function_Calling_with_OpenAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Integrating tools/function with LLM using OpenAI

### Why function calling?

When working with generative text models, it can be difficult to coerce generative models to give consistent outputs in a structured format such as JSON. Function Calling in Gemini allows you to overcome this limitation by forcing the model to output structured data in the format and schema that you define.

You can think of Function Calling as a way to get structured output from user prompts and function definitions, use that structured output to make an API request to an external system, then return the function response to the generative model so that it can generate a natural language summary. In other words, function calling in Gemini helps you go from unstructured text in prompt, to a structured data object, and back to natural language again.

### Benefits of Function Calling

- **Native framework**: Function Calling is a native framework in Gemini, so there's no need to enable additional APIs or install extra packages.
- **Time savings**: No need to write custom code to call, parse, and synthesize information from multiple APIs.
- **Simplified interaction with generative AI models**: Gemini handles the complex task of understanding the user's intent, predicting function calls, extracting relevant function parameters, and generating natural language summaries.
- **Versatility**: Easily extend capabilities by adding more function calls for different APIs and tailoring it to your needs.


In [ ]:
!pip install openai --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00


In [ ]:
# Define constants for the Azure OpenAI API configuration
import os
os.environ['OPENAI_API_KEY'] = "YOUR_API_KEY"

In [ ]:
from openai import OpenAI
import json
import requests

client = OpenAI()
model_name = "gpt-3.5-turbo"

In an API call, you can describe functions and have the model intelligently choose to output a JSON object containing arguments to call one or many functions. The Chat Completions API does not call the function; instead, the model generates JSON that you can use to call the function in your code.

## Supported models
Not all model versions are trained with function calling data. Function calling is supported with the following models: gpt-4, gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-4-0613, gpt-3.5-turbo, gpt-3.5-turbo-0125, gpt-3.5-turbo-1106, and gpt-3.5-turbo-0613

In addition, parallel function calls is supported on the following models: gpt-4-turbo-preview, gpt-4-0125-preview, gpt-4-1106-preview, gpt-3.5-turbo-0125, and gpt-3.5-turbo-1106

In [ ]:
# Define the OpenWeatherMap API key
OWM_API_KEY = "29af1cea50a401d8e624eea4660b3f59"

def get_current_weather(location, unit="kelvin"):
    """
    Fetches the current weather information for a given location.

    Parameters:
    - location: str, the name of the location (e.g., "Paris").
    - unit: str, the unit of temperature (default is "kelvin").

    Returns:
    - str: JSON formatted string containing weather information.
    """
    # Construct the API request URL
    url = f"https://api.openweathermap.org/data/2.5/weather?q={location}&appid={OWM_API_KEY}"

    # Send the API request
    response = requests.get(url)

    # Parse the temperature and weather forecast from the response
    temp = response.json()['main']['temp']
    forecast = [response.json()['weather'][0]['main'], response.json()['weather'][0]['description']]

    # Create a dictionary with the weather information
    weather_info = {
        "location": location,
        "temperature": temp,
        "unit": 'Kelvin',
        "forecast": forecast
    }

    # Return the weather information as a JSON string
    return json.dumps(weather_info)

In [ ]:
get_current_weather("Paris")

'{"location": "Paris", "temperature": 291.17, "unit": "Kelvin", "forecast": ["Clear", "clear sky"]}'

In [ ]:
get_current_weather("Manila")

'{"location": "Manila", "temperature": 302.12, "unit": "Kelvin", "forecast": ["Rain", "light rain"]}'

In [ ]:
messages = [{"role":"user",'content':"what is AI?"}]
results = client.chat.completions.create(model = model_name, messages = messages)
print(results.model_dump_json(indent=2))

{
  "id": "chatcmpl-9jdO58mgJUIxSu9Pjsx89ohncVUxi",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "AI, or artificial intelligence, refers to the simulation of human intelligence processes by machines, typically computer systems. This includes processes such as learning, reasoning, problem-solving, perception, and language understanding. AI technologies can be used in a wide range of applications, from virtual assistants like Siri and Alexa to self-driving cars and predictive analytics.",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1720663193,
  "model": "gpt-3.5-turbo-0125",
  "object": "chat.completion",
  "service_tier": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 66,
    "prompt_tokens": 11,
    "total_tokens": 77
  }
}


In [ ]:

# Define tools/functions for the LLM to use
tools = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit","kelvin"]},
                    },
                    "required": ["location"],
                },
            },
        },

    ]

In [ ]:
def get_response(messages, tools, model=model_name):
    """
    Handles interaction with the LLM, including making function calls if needed.

    Parameters:
    - messages: list, the conversation history.
    - tools: list, the functions/tools available to the LLM.
    - model: str, the model name to use (default is model_name).

    Returns:
    - str: The response from the LLM or function.
    """
    # Create a chat completion with the given messages and tools
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice='auto',
        temperature=0.2,
    )
    #print(response.model_dump_json(indent=2))
    # Get the generated response and tool calls (if any)
    response = response.choices[0].message
    tool_calls = response.tool_calls

    try:
        if tool_calls:
            print("Making a function call")
            # Available functions
            available_functions = {
                "get_current_weather": get_current_weather,
            }
            print(tool_calls)

            # Extend conversation with assistant's reply
            messages.append(response)

            # Handle each function call
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": function_response,
                    }
                )

            # Get a new response from the model with the function response
            second_response = client.chat.completions.create(
                model=model_name,
                messages=messages,
            )
            return second_response
        else:
            return response.content
    except Exception as e:
        print("Error occurred", e)
        return response

In [ ]:
# Example usage of get_response function
messages = [{"role": "user", "content": "Provide a 2 line explanation for AI"}]
response = get_response(messages, tools)
print(response)

AI, or Artificial Intelligence, is the simulation of human intelligence processes by machines, especially computer systems. It involves the development of algorithms that enable machines to perform tasks that typically require human intelligence.


In [ ]:
messages = [{"role": "user", "content": "How is the weather in Tokyo today?"}]
response = get_response(messages, tools)
print(response.choices[0].message.content)

Making a function call
[ChatCompletionMessageToolCall(id='call_5YlkdEcRaGJVviUkcnz9VadL', function=Function(arguments='{"location":"Tokyo","unit":"celsius"}', name='get_current_weather'), type='function')]
The weather in Tokyo today is mostly cloudy with broken clouds, and the temperature is around 30.91°C.
